In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!rm insurance.csv
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv


In [ ]:
dataset = pd.read_csv('insurance.csv')
dataset.dropna()

In [ ]:
categorical = dataset.select_dtypes(include=['object', 'category']).columns.tolist()
numerical = dataset.select_dtypes(include=['int64', 'float64']).columns.tolist()

for col in categorical:
    dataset[col] = dataset[col].astype('category').cat.codes


print(categorical)
print(numerical)

In [ ]:
features_list = ['age', 'sex', 'bmi', 'children', 'smoker', 'region']

print(features_list)

In [ ]:
train_df = dataset.sample(frac=0.8, random_state=0)
test_df = dataset.drop(train_df.index)

def df_to_ds(src):
    features = src.copy()
    labels = src.pop('expenses')
    return features, labels

train_dataset, train_labels = df_to_ds(train_df)
test_dataset, test_labels= df_to_ds(test_df)

In [ ]:
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))

In [ ]:
model = keras.Sequential([
    normalizer,
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(16, activation='relu'),
    layers.Dense(1)
    ])

model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['mae', 'mse'])

model.fit(train_dataset, train_labels, epochs=200, verbose=1)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
